In [1]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import csv
import numpy as np
from pandas import Series, DataFrame
import plotly.graph_objs as go
from plotly import tools

In [2]:
forecast = pd.read_csv("C:\\Data\\Twitter\\Weather-forecast.csv", sep=',', delimiter=None)

In [3]:
forecast.head()

,Date,Weather,Temperature,Days From Pull
0,2016-12-07,3,11,0
1,2016-12-08,8,11,1
2,2016-12-09,8,9,2
3,2016-12-10,8,11,3
4,2016-12-11,1,7,4


# Bucket the weather codes

In [4]:
#  1: Sunny - 2: Cloudy
#  3: Light rain - 4: Heavy rain
#  5: Sleet/Hail - 6: Snow
#  7: Thunder - 8: Fog/Mist


forecast['Bucket_Weather'] = None

for i in range(len(forecast.index)):   # number of rows in our dataframe - updated when new data are uploaded
    if (forecast['Weather'][i] in {2, 3, 7, 8}) == True:
            forecast['Bucket_Weather'][i] = 2
    
    elif (forecast['Weather'][i] in {9, 10, 11, 12}) == True:
            forecast['Bucket_Weather'][i] = 3
    
    elif (forecast['Weather'][i] in {13, 14, 15}) == True:
            forecast['Bucket_Weather'][i] = 4
        
    elif (forecast['Weather'][i] in {16, 17, 18, 19, 20, 21}) == True:
            forecast['Bucket_Weather'][i] = 5
    
    elif (forecast['Weather'][i] in {22, 23, 24, 25, 26, 27}) == True:
            forecast['Bucket_Weather'][i] = 6

    elif (forecast['Weather'][i] in {28, 29, 30}) == True:
            forecast['Bucket_Weather'][i] = 7

    elif (forecast['Weather'][i] in {5, 6}) == True:
            forecast['Bucket_Weather'][i] = 8
    
    else:
           forecast['Bucket_Weather'][i] = 1 

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning:


A value is trying to be set on a c

In [5]:
forecast.tail()

,Date,Weather,Temperature,Days From Pull,Bucket_Weather
135,2017-01-03,3,2,0,2
136,2017-01-04,7,5,1,2
137,2017-01-05,1,2,2,1
138,2017-01-06,7,3,3,2
139,2017-01-07,8,8,4,2


In [6]:
accurate = ((forecast[(forecast['Days From Pull'] == 0)]))

# Reset the index so numbers are increasing by 1
accurate = accurate.reset_index(drop=True)
#accurate

In [7]:
# Header for plots
header = accurate.columns.values

In [8]:
npArray = np.array(accurate)
npArray.shape

(28L, 5L)

In [9]:
# Put the date in X, the rest of the parameter in Y
X = npArray[:,0]
Y = npArray[:,1:]

In [10]:
# Create temperature with the forecast the day before
Onedaybefore = (forecast[(forecast['Days From Pull'] == 1)])
Onedaybefore = Onedaybefore.reset_index(drop=True)
npArray1 = np.array(Onedaybefore)
X1 = npArray1[:,0]
Y1 = npArray1[:,1:]


In [11]:
# Create temperature with the forecast two days before
Twodaybefore = (forecast[(forecast['Days From Pull'] == 2)])
Twodaybefore = Twodaybefore.reset_index(drop=True)
npArray2 = np.array(Twodaybefore)
X2 = npArray2[:,0]
Y2 = npArray2[:,1:]


In [12]:
# Create temperature with the forecast three days before
Threedaybefore = (forecast[(forecast['Days From Pull'] == 3)])
Threedaybefore = Threedaybefore.reset_index(drop=True)
npArray3 = np.array(Threedaybefore)
X3 = npArray3[:,0]
Y3 = npArray3[:,1:]


In [13]:
# Create temperature with the forecast four days before
Fourdaybefore = (forecast[(forecast['Days From Pull'] == 4)])
Fourdaybefore = Fourdaybefore.reset_index(drop=True)
#npArray4 = np.array(Threedaybefore)
#X4 = npArray4[:,0]
#Y4 = npArray4[:,1:]

# Plot the temperatures as line

In [15]:
#init_notebook_mode()

In [14]:
# Save graph into plotly account
import plotly
plotly.tools.set_credentials_file(username='Laura_Foulquier', api_key='zehu3fat3Mfs6v3pTNMY')

In [15]:
f1 = 1

trace0 = go.Scatter(
    x = X,
    y = Y[:,f1],
    name = 'Temperature at noon',
    line = dict(
        color = 'orange',
        width = 3
    )
)

trace1 = go.Scatter(
    x = X1,
    y = Y1[:,f1],
    name = 'Temp 1 day prediction',
    line = dict(
        color = 'orange',
        width = 2,
        dash = 'dot'
    )
)

trace2 = go.Scatter(
    x = X2,
    y = Y2[:,f1],
    name = 'Temp 2 day prediction',
    line = dict(
        color = 'blue',
        width = 2,
        dash = 'dot'
    )
)

trace3 = go.Scatter(
    x = X3,
    y = Y3[:,f1],
    name = 'Temp 3 day prediction',
    line = dict(
        color = 'green',
        width = 0.5,
        dash = 'dot',
    )
)

layout = Layout(
    xaxis = dict(title = 'Date'),
    yaxis = dict(
        title = 'Temperature',
        range = [0,16]
    ),
    showlegend = True,
    font=dict(family='Old Standard TT, serif', size=16, color='#7f7f7f')
)

data = [trace0, trace1, trace2, trace3]

fig = dict(data=data, layout = layout)
#iplot(fig)

# Put the plot in the plotly account
py.plot(fig, filename = 'MetOffice-Temperature-Prediction')

u'https://plot.ly/~Laura_Foulquier/2'

# Work on forecast accuracy

In [16]:
# Create an accuracy DataFrame to see how accurate the forecast is 
accuracy = pd.DataFrame()
accuracy['Date'] = accurate['Date']
accuracy['1Day'] = None
accuracy['2Day'] = None
accuracy['3Day'] = None
accuracy['4Day'] = None
#accuracy

# Build accuracy table

In [17]:
#  Put a 1 if the forecast match the accurate weather
#  0 otherwise

for i in (range(len(forecast.index)/5)): 
    if (Onedaybefore['Bucket_Weather'][i] == accurate['Bucket_Weather'][i]) == True:
        accuracy['1Day'][i] = 1
    else:
        accuracy['1Day'][i] = 0
    
for i in (range(len(forecast.index)/5)): 
    if (Twodaybefore['Bucket_Weather'][i] == accurate['Bucket_Weather'][i]) == True:
        accuracy['2Day'][i] = 1
    else:
        accuracy['2Day'][i] = 0

for i in (range(len(forecast.index)/5)): 
    if (Threedaybefore['Bucket_Weather'][i] == accurate['Bucket_Weather'][i]) == True:
        accuracy['3Day'][i] = 1
    else:
        accuracy['3Day'][i] = 0 

for i in (range(len(forecast.index)/5)): 
    if (Fourdaybefore['Bucket_Weather'][i] == accurate['Bucket_Weather'][i]) == True:
        accuracy['4Day'][i] = 1
    else:
        accuracy['4Day'][i] = 0  
    
#accuracy

# Plot accuracy for one day forecast

In [20]:
# Create the percentages
#True1Day = float(accuracy.sum(axis = 0)[1])/float(len(accuracy.index))
#False1Day = 1 - True1Day
#
#fig = {
#    'data': [{'labels': ['Exact forecast', 'Non matching forecast'],
#              'values': [True1Day, False1Day],
#              'type': 'pie'}],
#    'layout': {
#        'title': 'Accuracy of one day forecast',
#        'height': 400
#    }
#}
#
#iplot(fig)

# Grouped bar charts for all forecast

In [18]:
# Compute the percentages for the other forecasts

True1Day = accuracy.sum(axis = 0)[1]
False1Day = len(accuracy.index) - True1Day

True2Day = accuracy.sum(axis = 0)[2]
False2Day = len(accuracy.index) - True2Day

True3Day = accuracy.sum(axis = 0)[3]
False3Day = len(accuracy.index) - True3Day

True4Day = accuracy.sum(axis = 0)[4]
False4Day = len(accuracy.index)- True4Day

In [19]:
trace1 = go.Bar(
    x = ['1 Day Forecast', '2 Day Forecast', '3 Day Forecast', '4 Day Forecast'],
    y = [True1Day, True2Day, True3Day, True4Day],
    name = 'Accurate Weather Forecast',
    marker = dict(
        color = 'rgb(158,202,225)',
        line = dict(
            color = 'rgb(8,48,107)',
            width = 1.5)
    )
)

trace2 = go.Bar(
    x = ['1 Day Forecast', '2 Day Forecast', '3 Day Forecast', '4 Day Forecast'],
    y = [False1Day, False2Day, False3Day, False4Day],
    name = 'Non matching Weather Forecast',
    marker = dict(
        color = 'rgb(255,100,100)',
        line = dict(
            color = 'rgb(255,0,0)',
            width = 1.5)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode = 'group',
    height = 500,
    xaxis=dict(tickangle=-45),
    font=dict(family='Old Standard TT, serif', size=16, color='#7f7f7f'),
    yaxis = dict(
        title = 'Counts'
))

fig = go.Figure(data=data, layout=layout)

# Put the plot in the plotly account
py.plot(fig, filename = 'MetOffice-Weather-Accuracy')

u'https://plot.ly/~Laura_Foulquier/4'

# Export MetOffice Data for plotting with  tweets results

In [31]:
final_weather = accurate
final_weather = final_weather.drop(['Weather','Days From Pull'], axis=1)

In [35]:
final_weather.to_csv(path_or_buf='C:\\Data\\Twitter\\Final_weather_for_plotting.csv', sep=',', na_rep='null', \
                     header=True, index=False, index_label=None, mode='w', encoding=None, compression=None,\
                     quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False,\
                     date_format=None, doublequote=True, escapechar=None, decimal='.')